In [ ]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="Insira sua lcienca aqui") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 37.7 MB/s eta 0:00:00
Please provide a valid license UUID. You can use a free https://ampl.com/ce license.
AMPL License UUID (you can use free https://ampl.com/ce or https://ampl.com/courses licenses):
AMPL Version 20250421 (Linux-6.8.0-1025-azure, 64-bit)
Demo license with maintenance expiring 20270131.
Using license file "/usr/local/lib/python3.11/dist-packages/ampl_module_base/bin/ampl.lic".



#Listas

### Aula 10/04/2025

### Problema

Uma empresa deseja criar um planejamento de produção de notebooks para os próximos 4 trimestres.

- **Estoque inicial**: 5000 notebooks  
- **Demandas trimestrais esperadas**: [7000, 15000, 10000, 8000]  

- **Capacidade de produção normal por trimestre**: até 10.000 notebooks  
  - Custo por notebook: **R$ 1200,00**

- **Capacidade de produção extra por trimestre (horas-extras)**: até 2.500 notebooks  
  - Custo por notebook extra: **R$ 1500,00**

- É possível estocar notebooks produzidos para atender à demanda futura.  
  - Custo de armazenamento por notebook por trimestre: **R$ 100,00**

### Objetivo
Satisfazer a demanda total dos 4 trimestres com o **menor custo possível**, considerando produção normal, horas-extras e armazenamento.

### Modelo
Formule como um modelo de **Programação Linear Inteira Mista (PLIM)** e implemente usando **JuMP (Julia)** ou **Pyomo/AMPL (Python)**.


In [ ]:
# Ex 1

import time

# Captura o tempo inicial
inicio = time.time()

################## MODELO ###############################
conteudo_ampl = """
set T;

param DemandaTrimestral{T};
param CapacidadeTri;
param CapacidadeTriExtra;
param CustoNotebook;
param CustoNotebookExtra;
param CustoArmazenamento;
param EstoqueInicial;

# Variáveis de decisão
var ProduzNormal{T} >= 0;
var ProduzExtra{T} >= 0;
var Estoque{T} >= 0;

# Função objetivo: minimizar custo total
minimize CustoTotal:
    sum{t in T} (
        CustoNotebook * ProduzNormal[t] +
        CustoNotebookExtra * ProduzExtra[t] +
        CustoArmazenamento * Estoque[t]
    );

# Restrição: Limite de produção normal por trimestre
subject to LimiteNormal{t in T}:
    ProduzNormal[t] <= CapacidadeTri;

# Restrição: Limite de produção extra por trimestre
subject to LimiteExtra{t in T}:
    ProduzExtra[t] <= CapacidadeTriExtra;

# Restrição: Balanço do estoque (para t = 1)
subject to EstoqueInicialBalanco:
    Estoque[1] = EstoqueInicial + ProduzNormal[1] + ProduzExtra[1] - DemandaTrimestral[1];

# Restrição: Balanço do estoque (para t > 1)
subject to BalancoEstoque{t in {2,3,4}}:
    Estoque[t] = Estoque[t-1] + ProduzNormal[t] + ProduzExtra[t] - DemandaTrimestral[t];

# Restrição: Demanda de cada Tri
subject to DemandaInicial:
     Estoque[1] + ProduzNormal[1] + ProduzExtra[1] >= DemandaTrimestral[1];

# Restrição: Demanda de cada Tri
subject to DemandaTri{t in {2,3,4}}:
    Estoque[t-1] + ProduzNormal[t] + ProduzExtra[t] >= DemandaTrimestral[t];

"""

with open("/content/modelo.ampl", "w") as file:
    file.write(conteudo_ampl)

################## DADOS ###############################
conteudo_ampl = """
# Índice de trimestres
set T := 1 2 3 4;

# Demanda por trimestre
param DemandaTrimestral :=
1 7000
2 15000
3 10000
4 8000;

# Capacidade de produção normal por trimestre
param CapacidadeTri := 10000;

# Capacidade de produção com horas-extras por trimestre
param CapacidadeTriExtra := 2500;

# Custos
param CustoNotebook := 1200;
param CustoNotebookExtra := 1500;
param CustoArmazenamento := 100;

# Estoque inicial
param EstoqueInicial := 5000;
"""

with open("/content/dados.dat", "w") as file:
    file.write(conteudo_ampl)

################## SOLUÇÃO ###############################

from amplpy import AMPL

# Inicializar o AMPL
ampl = AMPL()

# Carregar arquivos
ampl.read("/content/modelo.ampl")
ampl.readData("/content/dados.dat")

# Configurar solver
ampl.setOption('solver', 'cplex')

# Resolver
ampl.solve()

# Mostrar custo total
custo_total = ampl.getObjective('CustoTotal').value()
print(f"Custo total mínimo: R$ {custo_total:,.2f}")

# Mostrar variáveis de decisão
produz_normal = ampl.getVariable("ProduzNormal")
produz_extra = ampl.getVariable("ProduzExtra")
estoque = ampl.getVariable("Estoque")

print("\nPlanejamento por trimestre:")
print("Trimestre | Produção Normal | Produção Extra | Estoque Final")
for t in ampl.getSet("T"):
    pn = produz_normal[t].value()
    pe = produz_extra[t].value()
    e = estoque[t].value()
    print(f"    {t}     |     {int(pn):5}        |     {int(pe):5}      |     {int(e):5}")


# Captura o tempo final
fim = time.time()

# Calcula o tempo decorrido
tempo_decorrido = fim - inicio
print(f"Tempo de execução: {tempo_decorrido:.2f} segundos")


CPLEX 22.1.2: CPLEX 22.1.2: optimal solution; objective 42500000
6 simplex iterations
Custo total mínimo: R$ 42,500,000.00

Planejamento por trimestre:
Trimestre | Produção Normal | Produção Extra | Estoque Final
    1     |      7000        |         0      |      5000
    2     |     10000        |         0      |         0
    3     |     10000        |         0      |         0
    4     |      8000        |         0      |         0
Tempo de execução: 0.13 segundos


## TC 2 Parte 1: Otimização Linear - Prof. Leduino

## Aviso Importante

**Importante**: Isto é um exercício didático da disciplina, de forma alguma é um incentivo à aposta na loteria. A probabilidade de acertar na loteria é, de qualquer forma, muito, muito pequena. Calcule!

## Objetivo do Exercício

Suponha que seu grupo foi contratado para fazer um modelo de otimização linear que encontre uma aposta de n números na mega-sena tal que:

1. O(a) apostador(a) entra inicialmente com um número da sorte escolhido arbitrariamente por ele/ela.

2. O modelo prioriza a escolha dos demais n-1 números menos sorteados considerando o histórico dos sorteios até o dia 7 de abril de 2025 (dados anexos).

3. A diferença entre a quantidade de números pares e a quantidade de números ímpares seja no máximo 2, considerando também o número da sorte.

4. A diferença entre a quantidade de números na 1ª metade superior e na 2ª metade inferior do bilhete seja no máximo 2, considerando também o número da sorte.

## Informações Adicionais

- Informações sobre a mega-sena: [Site Oficial da Caixa](https://loterias.caixa.gov.br/Paginas/mega-sena.aspx)

## Tarefa Adicional

2. [Apenas para os grupos com estudantes de pós-graduação] Use o Python para encontrar uma aposta de 6, 7, 8, 9 e 10 números.

### Solução sem PL

In [ ]:
# Importações necessárias:
import pandas as pd
import datetime
import itertools
import math

# 1. Ler o arquivo e filtrar dados até 07/04/2025:
arquivo = 'Mega-Sena.xlsx'
df = pd.read_excel(arquivo)

# Converter a coluna de data para datetime (ajuste o nome da coluna se necessário)
df['Data do Sorteio'] = pd.to_datetime(df['Data do Sorteio'], dayfirst=True)

# Filtrar os sorteios até 7 de abril de 2025
data_limite = datetime.datetime(2025, 4, 7)
df_filtrado = df[df['Data do Sorteio'] <= data_limite]

# 2. Calcular as frequências dos números de 1 a 60.
# As colunas que contém os números sorteados, por exemplo, 'Bola1' até 'Bola6'
colunas_bolas = ['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5', 'Bola6']

# Inicializa um dicionário para as contagens
frequencias = {i: 0 for i in range(1, 61)}

# Percorre as colunas e soma as ocorrências
for coluna in colunas_bolas:
    for num in df_filtrado[coluna]:
        if 1 <= num <= 60:
            frequencias[num] += 1

# 3. Função para verificar os critérios do bilhete:
def verifica_bilhete(bilhete):
    # bilhete é uma lista de números
    # Critério 1: diferença entre quantidade de pares e ímpares ≤ 2
    pares = sum(1 for num in bilhete if num % 2 == 0)
    impares = len(bilhete) - pares
    if abs(pares - impares) > 2:
        return False

    # Critério 2: diferença entre a quantidade de números na metade inferior (1 a 30) e na superior (31 a 60) ≤ 2
    inferior = sum(1 for num in bilhete if 1 <= num <= 30)
    superior = len(bilhete) - inferior
    if abs(inferior - superior) > 2:
        return False

    return True

# 4. Função para gerar aposta
def gera_aposta(n, numero_sorte, frequencias, pool_size=20):
    # n: quantidade total de números na aposta (6, 7, 8, 9 ou 10)
    # numero_sorte: número escolhido pelo apostador. Deve estar entre 1 e 60.
    if numero_sorte < 1 or numero_sorte > 60:
        raise ValueError("Número da sorte deve ser entre 1 e 60")

    # Cria a lista de candidatos descartando o número da sorte
    candidatos = [num for num in range(1, 61) if num != numero_sorte]

    # Ordena os candidatos por frequência (menor frequência primeiro)
    candidatos.sort(key=lambda x: frequencias[x]) # Nota de Debug: Conferi com excel, está correta a ordem.

    # Opcionalmente, reduzir pool para os (pool_size) números menos sorteados
    pool = candidatos[:pool_size]

    melhor_bilhete = None
    melhor_soma_freq = None

    # Gerar combinações de tamanho n-1 (pois o número da sorte já será incluído)
    for combinacao in itertools.combinations(pool, n - 1):
        bilhete = list(combinacao) + [numero_sorte]
        bilhete.sort()  # ordenar para exibição
        if verifica_bilhete(bilhete):
            soma_freq = sum(frequencias[num] for num in bilhete)
            if melhor_bilhete is None or soma_freq < melhor_soma_freq:
                melhor_bilhete = bilhete
                melhor_soma_freq = soma_freq

    return melhor_bilhete

# 5. Função para calcular a chance de ganhar
def chance_ganhar(n):
    return math.comb(60, 6) / math.comb(n, 6)

# 6. Execução: Escolher o número da sorte e os tamanhos de aposta desejados.
# Exemplo: número da sorte e apostas de 6, 7, 8, 9 e 10 números.
numero_sorte = 11  # Altere este valor conforme a escolha do apostador.
tamanhos_aposta = [6, 7, 8, 9, 10]

for n in tamanhos_aposta:
    bilhete = gera_aposta(n, numero_sorte, frequencias, pool_size=20)
    if bilhete is not None:
        print(f"\nAposta com {n} números (incluindo o número da sorte {numero_sorte}):")
        print(sorted(bilhete))
        print(f"Chance de ganhar: 1 em {int(chance_ganhar(n)):,d}")
    else:
        print(f"\nNão foi possível gerar uma aposta válida para {n} números com o número da sorte {numero_sorte}.")




Aposta com 6 números (incluindo o número da sorte 11):
[11, 15, 21, 26, 48, 55]
Chance de ganhar: 1 em 50,063,860

Aposta com 7 números (incluindo o número da sorte 11):
[11, 15, 21, 26, 40, 48, 55]
Chance de ganhar: 1 em 7,151,980

Aposta com 8 números (incluindo o número da sorte 11):
[11, 15, 21, 22, 26, 31, 48, 55]
Chance de ganhar: 1 em 1,787,995

Aposta com 9 números (incluindo o número da sorte 11):
[11, 15, 21, 22, 26, 31, 40, 48, 55]
Chance de ganhar: 1 em 595,998

Aposta com 10 números (incluindo o número da sorte 11):
[3, 11, 15, 21, 22, 26, 31, 40, 48, 55]
Chance de ganhar: 1 em 238,399


### Solução com PL

In [ ]:
#!pip install pulp

import pandas as pd
import datetime
import math
import pulp

# 1. Ler e filtrar os dados (supondo que seu arquivo "Mega-Sena.xlsx" esteja no Colab):
arquivo = 'Mega-Sena.xlsx'
df = pd.read_excel(arquivo)

# Converter a coluna "Data do Sorteio" para datetime, lembrando que as datas estão no formato dia/mês/ano
df['Data do Sorteio'] = pd.to_datetime(df['Data do Sorteio'], dayfirst=True)
data_limite = datetime.datetime(2025, 4, 7)
df_filtrado = df[df['Data do Sorteio'] <= data_limite]

# 2. Calcular as frequências dos números de 1 a 60 usando as colunas das bolas:
colunas_bolas = ['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5', 'Bola6']
frequencias = {i: 0 for i in range(1, 61)}
for coluna in colunas_bolas:
    for num in df_filtrado[coluna]:
        if 1 <= num <= 60:
            frequencias[num] += 1

# 3. Modelagem com PuLP
def gera_aposta_pulp(n, numero_sorte, frequencias):
    # Cria o modelo de minimização
    modelo = pulp.LpProblem("Aposta_MegaSena", pulp.LpMinimize)

    # Variáveis: x[i] = 1 se número i for escolhido, 0 caso contrário.
    x = {i: pulp.LpVariable(f"x_{i}", cat="Binary") for i in range(1, 61)}

    # Objetivo: minimizar a soma das frequências dos números selecionados.
    modelo += pulp.lpSum(frequencias[i] * x[i] for i in range(1, 61))

    # Restrição 1: escolher exatamente n números.
    modelo += pulp.lpSum(x[i] for i in range(1, 61)) == n, "TotalNumerosSelecionados"

    # Restrição 2: garantir que o número da sorte seja escolhido.
    if numero_sorte < 1 or numero_sorte > 60:
        raise ValueError("O número da sorte deve ser entre 1 e 60.")
    modelo += x[numero_sorte] == 1, "NumeroSorte"

    # Restrição 3: Diferença entre quantidade de pares e ímpares no máximo 2.
    soma_pares = pulp.lpSum(x[i] for i in range(1, 61) if i % 2 == 0)
    soma_impares = pulp.lpSum(x[i] for i in range(1, 61) if i % 2 != 0)
    modelo += (soma_pares - soma_impares) <= 2, "ParImpar_Pos"
    modelo += (soma_impares - soma_pares) <= 2, "ParImpar_Neg"

    # Restrição 4: Diferença entre números da primeira metade (1 a 30) e da segunda metade (31 a 60) no máximo 2.
    soma_inferior = pulp.lpSum(x[i] for i in range(1, 31))
    soma_superior = pulp.lpSum(x[i] for i in range(31, 61))
    modelo += (soma_inferior - soma_superior) <= 2, "Metade_Pos"
    modelo += (soma_superior - soma_inferior) <= 2, "Metade_Neg"

    # Resolve o modelo
    status = modelo.solve()

    if status != pulp.LpStatusOptimal:
        return None, None  # Não encontrou solução ótima

    # Obtém a solução: os números onde x[i] == 1
    bilhete = [i for i in range(1, 61) if pulp.value(x[i]) == 1]
    soma_freq = sum(frequencias[i] for i in bilhete)

    return bilhete, soma_freq

# 4. Função para calcular a chance de ganhar
def chance_ganhar(n):
    return math.comb(60, 6) / math.comb(n, 6)

# 5. Execução: exemplo com diferentes tamanhos de aposta e um número da sorte escolhido
numero_sorte = 11  # Altere conforme a escolha do apostador.
tamanhos_aposta = [6, 7, 8, 9, 10]

for n in tamanhos_aposta:
    bilhete, soma_freq = gera_aposta_pulp(n, numero_sorte, frequencias)
    if bilhete is not None:
        bilhete.sort()
        print(f"\nAposta com {n} números (incluindo o número da sorte {numero_sorte}):")
        print("Bilhete:", bilhete)
        print("Soma das frequências:", soma_freq)
        print("Chance de ganhar: 1 em {:,}".format(chance_ganhar(n)))
    else:
        print(f"\nNão foi possível gerar uma aposta válida para {n} números com o número da sorte {numero_sorte}.")


ModuleNotFoundError: No module named 'pulp'

In [ ]:
!pip install pulp
import pulp

# Criação do modelo
model = pulp.LpProblem("Redução_de_Contaminantes", pulp.LpMinimize)

# Variáveis de decisão
x11 = pulp.LpVariable("x11", 0, 1)  # Medida 1 - Altos Fornos
x21 = pulp.LpVariable("x21", 0, 1)  # Medida 2 - Altos Fornos
x31 = pulp.LpVariable("x31", 0, 1)  # Medida 3 - Altos Fornos

y12 = pulp.LpVariable("y12", 0, 1)  # Medida 1 - Fornos Abertos
y22 = pulp.LpVariable("y22", 0, 1)  # Medida 2 - Fornos Abertos
y32 = pulp.LpVariable("y32", 0, 1)  # Medida 3 - Fornos Abertos

# Função objetivo
model += (
    8 * x11 + 7 * x21 + 11 * x31 +
    10 * y12 + 6 * y22 + 9 * y32
)

# Restrições de redução mínima
model += (
    12 * x11 + 25 * x21 + 17 * x31 +
    9 * y12 + 20 * y22 + 13 * y32 >= 60, "Partículas"
)

model += (
    35 * x11 + 18 * x21 + 56 * x31 +
    42 * y12 + 31 * y22 + 49 * y32 >= 150, "ÓxidoSulfúrico"
)

model += (
    37 * x11 + 28 * x21 + 29 * x31 +
    53 * y12 + 34 * y22 + 20 * y32 >= 125, "Hidrocarbonetos"
)

# Resolver
model.solve()

# Resultados
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Custo Total: ${pulp.value(model.objective):.2f} milhões")

for var in model.variables():
    print(f"{var.name} = {var.varValue:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.6 MB/s eta 0:00:00
Status: Optimal
Custo Total: $31.46 milhões
x11 = 1.0000
x21 = 0.2884
x31 = 0.2806
y12 = 0.3356
y22 = 1.0000
y32 = 1.0000


Aula dia 17/04

In [ ]:
# Instalar PuLP (só precisa fazer isso uma vez no Colab)
!pip install -q pulp

import pulp

# Parâmetros
l = 8  # número de linhas
c = l  # número de colunas

# Criar o modelo
model = pulp.LpProblem("Rainhas_Maximizadas", pulp.LpMaximize)

# Variáveis binárias x[i][j] = 1 se há uma rainha na posição (i, j)
x = [[pulp.LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(c)] for i in range(l)]

# Função objetivo
model += pulp.lpSum(((i + j) % 2 == 0 and 1 or 2) * x[i][j] for i in range(l) for j in range(c))

# Restrição: uma única rainha por coluna
for j in range(c):
    model += pulp.lpSum(x[i][j] for i in range(l)) <= 1

# Restrição: uma única rainha por linha
for i in range(l):
    model += pulp.lpSum(x[i][j] for j in range(c)) <= 1

# Restrição: no máximo uma rainha por diagonal principal (j - i constante)
for k in range(-(c - 1), c):
    model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if j - i == k) <= 1

# Restrição: no máximo uma rainha por diagonal secundária (i + j constante)
for k in range(2 * c):
    model += pulp.lpSum(x[i][j] for i in range(l) for j in range(c) if i + j == k) <= 1

# Resolver o modelo
model.solve()

# Resultados
print("Status da otimização:", pulp.LpStatus[model.status])
print("Valor ótimo z =", pulp.value(model.objective))

# Mostrar o tabuleiro com rainhas
print("\nTabuleiro com Rainhas:")
for i in range(l):
    row = ""
    for j in range(c):
        row += "♛ " if pulp.value(x[i][j]) == 1 else ". "
    print(row)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.0 MB/s eta 0:00:00
Status da otimização: Optimal
Valor ótimo z = 12.0

Tabuleiro com Rainhas:
. ♛ . . . . . . 
. . . . . . ♛ . 
. . . . ♛ . . . 
♛ . . . . . . . 
. . . . . . . . 
. . . ♛ . . . . 
. . . . . ♛ . . 
. . ♛ . . . . . 


# Trabalho

In [ ]:
import pulp

# Cria o problema de maximização
model = pulp.LpProblem("Situacao1", pulp.LpMaximize)

# Define as variáveis de decisão
E1 = pulp.LpVariable("E1", lowBound=0)
E2 = pulp.LpVariable("E2", lowBound=0)
E3 = pulp.LpVariable("E3", lowBound=0)
E4 = pulp.LpVariable("E4", lowBound=0)
E5 = pulp.LpVariable("E5", lowBound=0)

# Função objetivo
model += (
    404.39999 * E1 +
    286.5 * E2 +
    183.3 * E3 +
    102.57 * E4 +
    127 * E5,
    "Lucro_Total"
)

# Restrições
model += E1 <= 21.66, "Restrição_E1"
model += E2 <= 14.17, "Restrição_E2"
model += E4 <= 6.34,  "Restrição_E4"
model += E5 <= 7.83,  "Restrição_E5"
# A restrição de E3 está faltando operador, então estou assumindo um limite superior arbitrário alto
model += E3 <= 11.33, "Restrição_E3"

model += (
    50.57 * E1 +    53.08 * E2 +    43.09 * E3 +    40.71 * E4 +    38.69 * E5 >= 2300,
    "Restrição_Minima_1"
)

model += (
    87.5 * E1 +    68.3 * E2 +    58 * E3 +    56.5 * E4 +    48.6 * E5 <= 3500,
    "Restrição_Maxima_2"
)

model += (
    6.7 * E1 +    6.6 * E2 +    4.8 * E3 +    4.5 * E4 +    5.6 * E5 <= 310,
    "Restrição_Maxima_3"
)

# Resolve o modelo
model.solve()

# Exibe os resultados
print("Status:", pulp.LpStatus[model.status])
for var in model.variables():
    print(f"{var.name} = {var.varValue}")
print("Lucro Total =", pulp.value(model.objective))


Status: Optimal
E1 = 21.66
E2 = 14.17
E3 = 10.981707
E4 = 0.0
E5 = 0.0
Lucro Total = 14831.955676500002


In [ ]:
# Valores das variáveis da solução
E1 = 21.66
E2 = 14.17
E3 = 11.33
E4 = 4.716222
E5 = 0.0

# Checagem das restrições
def checar_restricoes(E1, E2, E3, E4, E5):
    violadas = []

    # Limites superiores individuais
    if E1 > 21.66: violadas.append("E1 > 21.66")
    if E2 > 14.17: violadas.append("E2 > 14.17")
    if E3 > 11.33: violadas.append("E3 > 511.33")
    if E4 > 6.34: violadas.append("E4 > 6.34")
    if E5 > 7.83: violadas.append("E5 > 7.83")

    # Restrição 1: mínimo
    expr1 = 50.57*E1 + 53.08*E2 + 43.09*E3 + 40.71*E4 + 38.69*E5
    if expr1 < 2300:
        violadas.append(f"Restriçao 1 violada: {expr1:.2f} < 2300")

    # Restrição 2: máximo
    expr2 = 87.5*E1 + 68.3*E2 + 58*E3 + 56.5*E4 + 48.6*E5
    if expr2 > 3500:
        violadas.append(f"Restriçao 2 violada: {expr2:.2f} > 3500")

    # Restrição 3: máximo
    expr3 = 6.7*E1 + 6.6*E2 + 4.8*E3 + 4.5*E4 + 5.6*E5
    if expr3 > 310:
        violadas.append(f"Restriçao 3 violada: {expr3:.2f} > 310")

    # Resultado
    if not violadas:
        print("✅ Nenhuma restrição foi violada.")
    else:
        print("⚠️ Restrições violadas:")
        for v in violadas:
            print("-", v)

# Executa a checagem
checar_restricoes(E1, E2, E3, E4, E5)


⚠️ Restrições violadas:
- Restriçao 2 violada: 3786.67 > 3500
- Restriçao 3 violada: 314.25 > 310


In [ ]:
import pulp

# Criação do problema de maximização
model = pulp.LpProblem("Situacao2", pulp.LpMaximize)

# Definindo variáveis de decisão (contínuas, >= 0)
E1 = pulp.LpVariable("E1", lowBound=0)
E2 = pulp.LpVariable("E2", lowBound=0)
E3 = pulp.LpVariable("E3", lowBound=0)
E4 = pulp.LpVariable("E4", lowBound=0)
E5 = pulp.LpVariable("E5", lowBound=0)

# Função objetivo
model += 404.39999 * E1 + 286.5 * E2 + 183.3 * E3 + 102.57 * E4 + 127 * E5, "Lucro_Total"

# Restrições
model += E1 <= 21.66, "Restricao_2"
model += E2 <= 14.17, "Restricao_3"
model += E3 <= 11.33, "Restricao_4"
model += E4 <= 6.34, "Restricao_5"
model += E5 <= 7.83, "Restricao_6"

model += 50.57 * E1 + 53.08 * E2 + 43.09 * E3 + 40.71 * E4 + 38.69 * E5 >= 2600, "Restricao_7"
model += 67.5 * E1 + 68.3 * E2 + 58 * E3 + 56.5 * E4 + 48.6 * E5 <= 3700, "Restricao_8"
model += 6.7 * E1 + 6.6 * E2 + 4.8 * E3 + 4.5 * E4 + 5.6 * E5 <= 350, "Restricao_9"

# Resolver o modelo
model.solve()

# Exibir resultados
print("Status:", pulp.LpStatus[model.status])
for var in model.variables():
    print(f"{var.name} = {var.varValue:.4f}")
print(f"Lucro Total = {pulp.value(model.objective):.4f}")


Status: Optimal
E1 = 21.6600
E2 = 14.1700
E3 = 11.3300
E4 = 6.3400
E5 = 5.0789
Lucro Total = 16191.1155


In [ ]:
import pulp

# Criação do problema de maximização
model = pulp.LpProblem("Situacao3", pulp.LpMaximize)

# Definindo variáveis de decisão (contínuas, >= 0)
E1 = pulp.LpVariable("E1", lowBound=0)
E2 = pulp.LpVariable("E2", lowBound=0)
E3 = pulp.LpVariable("E3", lowBound=0)
E4 = pulp.LpVariable("E4", lowBound=0)
E5 = pulp.LpVariable("E5", lowBound=0)

# Função objetivo
model += 486.5 * E1 + 301.20001 * E2 + 195.39999 * E3 + 110.7 * E4 + 131.2 * E5, "Lucro_Total"

# Restrições
model += E1 <= 21.66, "Restricao_2"
model += E2 <= 14.17, "Restricao_3"
model += E3 <= 11.33, "Restricao_4"
model += E4 <= 6.34, "Restricao_5"
model += E5 <= 7.83, "Restricao_6"

model += 49.63 * E1 + 51.5 * E2 + 39.7 * E3 + 36.7 * E4 + 34.9 * E5 >= 1700, "Restricao_7"
model += 275 * E1 + 170 * E2 + 105 * E3 + 68 * E4 + 71 * E5 >= 9250, "Restricao_8"
model += 67.5 * E1 + 68.3 * E2 + 58 * E3 + 56.5 * E4 + 48.6 * E5 <= 3500, "Restricao_9"
model += 6.5 * E1 + 6 * E2 + 4.3 * E3 + 3.9 * E4 + 5.4 * E5 <= 450, "Restricao_10"

# Resolver o modelo
model.solve()

# Exibir resultados
print("Status:", pulp.LpStatus[model.status])
for var in model.variables():
    print(f"{var.name} = {var.varValue:.4f}")
print(f"Lucro Total = {pulp.value(model.objective):.4f}")


Status: Optimal
E1 = 21.6600
E2 = 14.1700
E3 = 11.3300
E4 = 0.5745
E5 = 7.8300
Lucro Total = 18110.3726


In [ ]:
# Solução dada
E1 = 21.66
E2 = 14.17
E3 = 11.33
E4 = 0.832583
E5 = 7.53

# Restrições
restricoes = {
    "R2: E1 <= 21.66": E1 <= 21.66,
    "R3: E2 <= 14.17": E2 <= 14.17,
    "R4: E3 <= 11.33": E3 <= 11.33,
    "R5: E4 <= 6.34": E4 <= 6.34,
    "R6: E5 <= 7.83": E5 <= 7.83,
    "R7: 49.63E1 + 51.5E2 + 39.7E3 + 36.7E4 + 34.9E5 >= 1700":
        49.63 * E1 + 51.5 * E2 + 39.7 * E3 + 36.7 * E4 + 34.9 * E5 >= 1700,
    "R8: 275E1 + 170E2 + 105E3 + 68E4 + 71E5 >= 9250":
        275 * E1 + 170 * E2 + 105 * E3 + 68 * E4 + 71 * E5 >= 9250,
    "R9: 67.5E1 + 68.3E2 + 58E3 + 56.5E4 + 48.6E5 <= 3500":
        67.5 * E1 + 68.3 * E2 + 58 * E3 + 56.5 * E4 + 48.6 * E5 <= 3500,
    "R10: 6.5E1 + 6E2 + 4.3E3 + 3.9E4 + 5.4E5 <= 450":
        6.5 * E1 + 6 * E2 + 4.3 * E3 + 3.9 * E4 + 5.4 * E5 <= 450,
}

# Verificação das restrições
print("Verificação das Restrições:\n")
for nome, condicao in restricoes.items():
    status = "✔️ OK" if condicao else "❌ VIOLADA"
    print(f"{nome}: {status}")


Verificação das Restrições:

R2: E1 <= 21.66: ✔️ OK
R3: E2 <= 14.17: ✔️ OK
R4: E3 <= 11.33: ✔️ OK
R5: E4 <= 6.34: ✔️ OK
R6: E5 <= 7.83: ✔️ OK
R7: 49.63E1 + 51.5E2 + 39.7E3 + 36.7E4 + 34.9E5 >= 1700: ✔️ OK
R8: 275E1 + 170E2 + 105E3 + 68E4 + 71E5 >= 9250: ✔️ OK
R9: 67.5E1 + 68.3E2 + 58E3 + 56.5E4 + 48.6E5 <= 3500: ✔️ OK
R10: 6.5E1 + 6E2 + 4.3E3 + 3.9E4 + 5.4E5 <= 450: ✔️ OK


In [ ]:
import pulp

# Criação do problema de maximização
model = pulp.LpProblem("Situacao4", pulp.LpMaximize)

# Definindo variáveis de decisão (contínuas, >= 0)
E1 = pulp.LpVariable("E1", lowBound=0)
E2 = pulp.LpVariable("E2", lowBound=0)
E3 = pulp.LpVariable("E3", lowBound=0)
E4 = pulp.LpVariable("E4", lowBound=0)
E5 = pulp.LpVariable("E5", lowBound=0)

# Função objetivo
model += 486.5 * E1 + 301.20001 * E2 + 195.39999 * E3 + 110.7 * E4 + 131.2 * E5, "Lucro_Total"

# Restrições
model += E1 <= 21.66, "Restricao_2"
model += E2 <= 14.17, "Restricao_3"
model += E3 <= 11.33, "Restricao_4"
model += E4 <= 6.34, "Restricao_5"
model += E5 <= 7.83, "Restricao_6"

model += 50.57 * E1 + 53.08 * E2 + 43.09 * E3 + 40.71 * E4 + 38.69 * E5 >= 2500, "Restricao_7"
model += 275 * E1 + 170 * E2 + 105 * E3 + 68 * E4 + 71 * E5 >= 8000, "Restricao_8"
model += 67.5 * E1 + 68.3 * E2 + 58 * E3 + 56.5 * E4 + 48.6 * E5 <= 4500, "Restricao_9"

# Resolver o modelo
model.solve()

# Exibir resultados
print("Status:", pulp.LpStatus[model.status])
for var in model.variables():
    print(f"{var.name} = {var.varValue:.4f}")
print(f"Lucro Total = {pulp.value(model.objective):.4f}")


Status: Optimal
E1 = 21.6600
E2 = 14.1700
E3 = 11.3300
E4 = 6.3400
E5 = 7.8300
Lucro Total = 18748.6100


In [ ]:
import pulp

# Criação do problema de maximização
model = pulp.LpProblem("situacao5", pulp.LpMaximize)

# Definindo variáveis de decisão (contínuas, >= 0)
E1 = pulp.LpVariable("E1", lowBound=0)
E2 = pulp.LpVariable("E2", lowBound=0)
E3 = pulp.LpVariable("E3", lowBound=0)
E4 = pulp.LpVariable("E4", lowBound=0)
E5 = pulp.LpVariable("E5", lowBound=0)

# Função objetivo
model += 498.29999 * E1 + 314.10001 * E2 + 207.8 * E3 + 223.5 * E4 + 142.60001 * E5, "Lucro_Total"

# Restrições
model += E1 <= 21.66, "Restricao_2"
model += E2 <= 14.17, "Restricao_3"
model += E3 <= 11.33, "Restricao_4"
model += E4 <= 6.34, "Restricao_5"
model += E5 <= 7.83, "Restricao_6"

model += 49.63 * E1 + 51.5 * E2 + 39.7 * E3 + 36.7 * E4 + 34.9 * E5 >= 2500, "Restricao_7"
model += 275 * E1 + 170 * E2 + 105 * E3 + 68 * E4 + 71 * E5 >= 8000, "Restricao_8"
model += 6.5 * E1 + 6 * E2 + 4.3 * E3 + 3.9 * E4 + 5.4 * E5 <= 400, "Restricao_9"

# Resolver o modelo
model.solve()

# Exibir resultados
print("Status:", pulp.LpStatus[model.status])
for var in model.variables():
    print(f"{var.name} = {var.varValue:.4f}")
print(f"Lucro Total = {pulp.value(model.objective):.4f}")


Status: Optimal
E1 = 21.6600
E2 = 14.1700
E3 = 11.3300
E4 = 6.3400
E5 = 7.8300
Lucro Total = 20131.8970
